In [1]:
import torch

In [2]:
import importlib
componets = 'slr_network.SLRModel'.rsplit('.')
mod = importlib.import_module(componets[0])
mod = getattr(mod, componets[1])
mod

slr_network.SLRModel

In [1]:
import numpy as np

In [2]:
import yaml, utils

sparser = utils.get_parser()
p = sparser.parse_args([])
if p.config is not None:
    with open(p.config, 'r') as f:
        try:
            default_arg = yaml.load(f, Loader=yaml.FullLoader)
        except AttributeError:
            default_arg = yaml.load(f)
    key = vars(p).keys()
    for k in default_arg.keys():
        if k not in key:
            print('WRONG ARG: {}'.format(k))
            assert (k in key)
    sparser.set_defaults(**default_arg)
args = sparser.parse_args([])
with open(f"./configs/{args.dataset}.yaml", 'r') as f:
    args.dataset_info = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
arg=args
gloss_dict = np.load(arg.dataset_info['dict_path'], allow_pickle=True).item()


In [8]:
if arg.load_checkpoints or arg.load_weights:
    load_slowfast_pkl = False
else:
    load_slowfast_pkl = True

In [4]:
import importlib
import torch.nn as nn
components = 'slr_network.SLRModel'.rsplit('.', 1)
mod = importlib.import_module(components[0])
mod = getattr(mod, components[1])


/home/nirmal/miniconda3/envs/slr_bf16/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nirmal/miniconda3/envs/slr_bf16/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
model = mod(
    num_classes=1296,
    c2d_type='resnet34',
    conv_type=2,
    use_bn=1,
    share_classifier=1,
    weight_norm=True,
    gloss_dict=gloss_dict,
    loss_weights=arg.loss_weights,
)

In [10]:
arg.loss_weights, arg.slowfast_config, arg.slowfast_args

({'SeqCTC': 1.0, 'ConvCTC': 1.0, 'Dist': 25.0},
 'SLOWFAST_64x2_R101_50_50.yaml',
 {})

In [6]:
model

SLRModel(
  (conv2d): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn1): Bat

In [7]:
args.model_args

{'num_classes': 1296,
 'c2d_type': 'resnet34',
 'conv_type': 2,
 'use_bn': 1,
 'share_classifier': True,
 'weight_norm': True,
 'use_graph': False}